In [ ]:
from IPython.display import display, Markdown as md
display(md(f"# Social Media Monitoring"))
display(md(f"**Search for a Politician**"))
display(md(f"***The following table contains data on the politicians and their associated social acounts***"))

In [ ]:
import os
from smm_wrapper import SMM
import qgrid
from ipywidgets import widgets, Output
from IPython.display import display, clear_output
smm = SMM()

politicians_df = smm.dv.politicians_df()
qgrid_widget = qgrid.show_grid(politicians_df)
qgrid_widget

In [ ]:
display(md(f"**Select a politician**"))
def on_button_clicked(b):
    global politician
    
    # use the out widget so the output is overwritten when two or more
    # searches are performed
    with out:
        try:
            politician = smm.api.politician_search(politician_id=searchTerm.value)
            clear_output()
            display('Result found for #: {}. Name and last name: {} {}'.format(politician['politician_id'], 
                                                                               politician['firstname'], 
                                                                               politician['name']))

        except:
            clear_output()
            display(md(f'The politician with id *"{searchTerm.value}"* was not found. Please enter a number between 1 and 2516'))

# create and display the button
searchTerm = widgets.Text()
button = widgets.Button(description="Search")
example = md("Please enter a politician id from the table above. Example: *2193 for Angela Merkel*")
display(example, searchTerm, button)

# the output widge is used to remove the output after the search field
out = Output()
display(out)

# set the event
button.on_click(on_button_clicked)

# trigger the event with the default value
on_button_clicked(button)

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RadioButtonGroup
output_notebook()
import pandas as pd
from datetime import datetime

In [ ]:
def modify_tw(doc):
    #getting the data from SMM api
    def extracting_data(politician_id, agg):
        filtered_tweets = smm.api.tweets_by(politician_id=politician_id, aggregate_by=agg)
        filtered_replies = smm.api.reply_to(politician_id=politician_id, aggregate_by=agg)
        filtered_posts = smm.api.posts_by(politician_id=politician_id, aggregate_by=agg)
        filtered_comments = smm.api.comments_by(politician_id=politician_id, aggregate_by=agg)
        filtered_tweets['labels'] = [datetime.strptime(item, '%Y-%m-%d') for item in filtered_tweets['labels']]
        filtered_replies['labels'] = [datetime.strptime(item, '%Y-%m-%d') for item in filtered_replies['labels']]
        filtered_posts['labels'] = [datetime.strptime(item, '%Y-%m-%d') for item in filtered_posts['labels']]
        filtered_comments['labels'] = [datetime.strptime(item, '%Y-%m-%d') for item in filtered_comments['labels']]
        source_tw = ColumnDataSource(data = dict(labels = filtered_tweets['labels'],values = filtered_tweets['values']))
        source_re = ColumnDataSource(data = dict(labels=filtered_replies['labels'],values=filtered_replies['values']))
        source_p = ColumnDataSource(data = dict(labels = filtered_posts['labels'],values = filtered_posts['values']))
        source_co = ColumnDataSource(data = dict(labels=filtered_comments['labels'],values=filtered_comments['values']))
        return(source_tw, source_re, source_p, source_co)
    
    #getting the data for the plots
    source_tw, source_re, source_p, source_co = extracting_data(politician['politician_id'], agg='month')  
    
    #creating plots
    #twitter plot
    t = figure(title="Aggregated tweets and replies by Politician ID", x_axis_type='datetime',
               plot_height=400, plot_width=700,
               background_fill_color='#efefef', x_axis_label='Date', y_axis_label='Tweets')
    t.line('labels', 'values', source = source_tw, legend="number of tweets", line_width=2, line_color="blue")
    t.line('labels', 'values', source = source_re, legend="number of replies", line_width=1, line_color="red")
    t.legend.location = "top_left"
    t.legend.click_policy="hide"
    #facebook posts plot
    p = figure(title="Aggregated facebook posts by Politician ID", x_axis_type='datetime',
               plot_height=400, plot_width=700,
               background_fill_color='#efefef', x_axis_label='Date', y_axis_label='Facebook posts')
    p.line('labels', 'values', source = source_p, legend="number of posts", line_width=2, line_color="blue")
    p.legend.location = "top_left"
    p.legend.click_policy="hide"
    #facebook comments plot
    c = figure(title="Aggregated facebook comments by Politician ID", x_axis_type='datetime',
               plot_height=400, plot_width=700,
               background_fill_color='#efefef', x_axis_label='Date', y_axis_label='Facebook comments')
    c.line('labels', 'values', source = source_co, legend="number of comments", line_width=2, line_color="green")
    c.legend.location = "top_left"
    c.legend.click_policy="hide"
    
    #radio button handler
    def callback(attr, old, new):
        if new == 0:
            agg = 'day'
        elif new == 1:
            agg = 'week'
        else:
            agg = 'month'
        source_tw_up, source_re_up, source_p_up, source_co_up = extracting_data(politician['politician_id'], agg=agg)
        source_tw.data, source_re.data, source_p.data, source_co.data = source_tw_up.data, source_re_up.data, source_p_up.data, source_co_up.data

        
    #choosing the aggregation
    radio_button_group = RadioButtonGroup(
            labels=["Daily", "Weekly", "Monthly"], active=2)
    radio_button_group.on_change('active', callback)
    

    doc.add_root(column(radio_button_group, t, p, c))

In [ ]:
show(modify_tw, notebook_url="http://10.6.13.139:8020")